In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from glob import glob

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import copy

import geopandas as gpd

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import gpflow
import tensorflow as tf
import sys
code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
from math import radians, cos, sin, asin, sqrt
from onoffgpf import OnOffSVGP, OnOffLikelihood

import pickle

from math import radians, cos, sin, asin, sqrt

import copy
import sys

import geopandas as gpd
import numpy as np

import gpflow


code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)

code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)

from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood
gpflow.config.default_float()


from zinf_gp.metrics import normcdf, fixed_top_X

2023-02-28 13:42:35.916269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 13:42:36.391741: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-28 13:42:36.643714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 13:42:36.643735: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:

data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

log_dir='/cluster/tufts/hugheslab/kheuto01/opioid/logs/fix_two/'

run_template = '{time}_{loc}_{model}_{start_year}_{cov}_{num_inducing}_{lr}'

In [26]:
times = ['annual', ]
locs = ['tract',]
start_years = [ 2000]
covs = [ '-auto',]
models = ['normal', ]
learning_rates = [0.1,0.01,0.05]
inducing_points = [400]

In [27]:
recalc_results = []
test_years = 2
geography_col='geoid'
timestep_col='timestep'
outcome_col='deaths'

town_map = pd.read_csv(os.path.join(data_dir,'town_tract_map.csv'), dtype=str)
group_map = gpd.read_file(os.path.join(data_dir, 'tract_group_map'), dtype=str)


# test y always comes from quarterly tract
y_timesteps_per_year = 4
file_name = f'clean_quarter_tract'
data_path = os.path.join(data_dir, file_name)

x_idx_cols = [geography_col, 'lat','lon', timestep_col,
      'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
      'svi_pctile',
      'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat','lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

data_gdf = gpd.read_file(data_path)

last_train_year = 2018


test_y = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][y_idx_cols]
starting_y_timestep = int(test_y[timestep_col].min())

sorted_y_timesteps = test_y[timestep_col].unique()
sorted_y_timesteps.sort()

for loc in locs:
    for time in times:
        x_timesteps_per_year = {'quarter':4, 'semi':2,'annual':1}[time]
        
        file_name = f'clean_{time}_{loc}'
        data_path = os.path.join(data_dir, file_name)

        data_gdf = gpd.read_file(data_path)

        test_x = data_gdf[(data_gdf['year'] > last_train_year) &
                          (data_gdf['year'] <= last_train_year+test_years)][x_idx_cols]
        
        starting_x_timestep = int(test_x[timestep_col].min())
        
        test_timesteps_per_year = max(y_timesteps_per_year, x_timesteps_per_year)
        test_timesteps = test_timesteps_per_year*test_years
        
        x_repeats = int(test_timesteps_per_year/x_timesteps_per_year)
        y_repeats = int(test_timesteps_per_year/y_timesteps_per_year)
        
        sorted_x_timesteps = test_x[timestep_col].unique()
        sorted_x_timesteps.sort()
        
        x_timesteps = [timestep  for timestep in sorted_x_timesteps for _ in range(x_repeats)]
        
        y_timesteps = [timestep  for timestep in sorted_y_timesteps for _ in range(y_repeats)]

        for start_year in start_years:
            print('HNY')
            
            for num_inducing in inducing_points:
                for model in models:
                    for cov in covs:
                    
                        best_elbo = -np.inf
                        bpr_2019 = np.NaN
                        bpr_2020 = np.NaN
                        
                        for lr in learning_rates:
                            print(lr)
                            this_run = run_template.format(time=time,loc=loc,
                                                           model=model,start_year=start_year,
                                                           cov=cov,
                                                           num_inducing=num_inducing,lr=lr)
                            try:
                                with open(os.path.join(log_dir,this_run,'model.mod'),'rb') as f:
                                        predictor = pickle.load(f)
                                with open(os.path.join(log_dir,this_run,'stats.csv'),'rb') as f:
                                    stats = pd.read_csv(f)
                                    elbo = stats.iloc[-1,:][['elbo']].values[0]
                                    
                            except(FileNotFoundError):
                                #print(f'Broke {this_run}')
                                data = [time, loc, start_year, cov, model, num_inducing, np.NaN, np.NaN]
                                recalc_results.append(data)
                                continue


                            xtops = []
                            for year in range(test_years):
                                xtop_year = []
                                max_timesteps = max(x_timesteps_per_year, y_timesteps_per_year)
                                for x_time,y_time in zip(x_timesteps[year*max_timesteps:(year+1)*max_timesteps],
                                                         y_timesteps[year*max_timesteps:(year+1)*max_timesteps]):

                                    test_x_time = test_x[test_x[timestep_col] == x_time]
                                    test_y_time = test_y[test_y[timestep_col] == y_time]
                                    _, _, _, fmean, fvar, gmean, gvar, _, _ = predictor.build_predict(test_x_time.loc[:, features_only].values)
                                    g_cond = tf.math.softplus(fmean * normcdf(gmean)).numpy()
                                    pred_df = pd.Series(g_cond.squeeze(), index=test_x_time[geography_col])
                                    
                                    if loc == 'town':
                                        merged_to_map = town_map.merge(pred_df.rename(outcome_col), right_index=True,left_on='parent_town', how='right')
                                        averaged_over_duplicates = merged_to_map.groupby('child_tracts').mean()[outcome_col]
                                        y_index = test_y[geography_col].unique()
                                        pred_df = pd.Series(index=y_index,dtype='float64')
                                        pred_df.update(averaged_over_duplicates)
                                        # not all tracts get mapped from towns
                                        pred_df=pred_df.fillna(0)
                                    elif loc == 'group':
                                        merged_to_map = group_map.merge(pred_df.rename(outcome_col), right_index=True,left_on='grouping', how='right')
                                        averaged_over_duplicates = merged_to_map.groupby('geoid').mean()[outcome_col]
                                        y_index = test_y[geography_col].unique()
                                        pred_df = pd.Series(index=y_index,dtype='float64')
                                        pred_df.update(averaged_over_duplicates)
                                        
                                    xtop_year.append(fixed_top_X(test_y_time.set_index(geography_col)[outcome_col], pred_df, 100))
                                xtops.append(xtop_year)
                            curr_results = {} 
                            for y, xtop in enumerate( xtops):
                                curr_results[f'bpr_100_{y}'] = copy.deepcopy(np.mean([thing[3] for thing in xtop]))
                                
                            bpr_2019 = copy.copy(curr_results[f'bpr_100_0'])
                            bpr_2020 = copy.copy(curr_results[f'bpr_100_1'])
                            print(bpr_2019)
                                
                            if elbo > best_elbo:
                                best_bpr_2019 = copy.copy(bpr_2019)
                                best_bpr_2020 = copy.copy(bpr_2020)
                                best_elbo = elbo
                                

                        data = [time, loc, start_year, cov, model, num_inducing, best_bpr_2019, best_bpr_2020]
                        recalc_results.append(data)

#        pd.DataFrame(recalc_results).to_csv(f'~/recalc_fix_results_sheet_{loc}_{time}.csv')
#pd.DataFrame(recalc_results).to_csv('~/recalc_fix_results_sheet.csv')

HNY
0.1
0.33628776780490743
0.01
0.322233869677604
0.05
0.35320447482645095


In [29]:
xtop_year

[(132.0, 0.2826086956521739, 132.0, 0.2826086956521739),
 (152.0, 0.3183856502242152, 152.0, 0.31838565022421517),
 (112.0, 0.3672316384180791, 112.0, 0.367231638418079),
 (139.0, 0.31862745098039214, 139.0, 0.318627450980392)]

In [11]:
fixed_top_X(test_y_time.set_index(geography_col)[outcome_col], pred_df, 100)

(142.0, 0.22826086956521738, 142.0, 0.22826086956521732)

In [20]:
xtops

[[(100.0, 0.4152046783625731, 100.0, 0.4152046783625731)],
 [(142.0, 0.22826086956521738, 142.0, 0.22826086956521732)]]

In [24]:
y_timesteps[year*y_timesteps_per_year:(year+1)*y_timesteps_per_year]

[80.0, 81.0, 82.0, 83.0]

In [37]:
xtops=[]
for year in range(1):
    xtop_year = []
    for x_time,y_time in zip(x_timesteps[year*max_timesteps:(year+1)*max_timesteps],
                             y_timesteps[year*max_timesteps:(year+1)*max_timesteps]):

        test_x_time = test_x[test_x[timestep_col] == x_time]
        test_y_time = test_y[test_y[timestep_col] == y_time]
        _, _, _, fmean, fvar, gmean, gvar, _, _ = predictor.build_predict(test_x_time.loc[:, features_only].values)
        g_cond = tf.math.softplus(fmean * normcdf(gmean)).numpy()
        pred_df = pd.Series(g_cond.squeeze(), index=test_x_time[geography_col])
        xtop_year.append(fixed_top_X(test_y_time.set_index(geography_col)[outcome_col], pred_df, 100))
    xtops.append(xtop_year)

In [53]:
result = fixed_top_X(test_y_time.set_index(geography_col)[outcome_col], pred_df, 100)[-1]

for i in range(1):
    new_result = debug_top_X(test_y_time.set_index(geography_col)[outcome_col], pred_df, 100)[-1]
    if new_result != result:
        print('did it!')

> /tmp/ipykernel_229437/1639764889.py(37)debug_top_X()
     35 
     36     import pdb;pdb.set_trace()
---> 37     bootstrapped_absolute = np.mean([np.abs(top_X_true.sum() - true_qtr_val[indices].sum())
     38                                      for indices in bootstrapped_all_indices])
     39     bootstrapped_ratio = np.mean([np.abs(true_qtr_val[indices]).sum() / np.abs(top_X_true).sum()

ipdb> num_tied_spots
1
ipdb> exit


In [ ]:
t

In [51]:
def debug_top_X(true_qtr_val, pred_qtr_val, X=10):
    top_X_predicted = pred_qtr_val.sort_values(ascending=False).iloc[:X]
    top_X_true = true_qtr_val.sort_values(ascending=False).iloc[:X]

    undisputed_top_predicted = top_X_predicted[top_X_predicted > top_X_predicted.min()]
    num_tied_spots = X - len(undisputed_top_predicted)
    undisputed_top_true = top_X_true[top_X_true > top_X_true.min()]
    num_true_ties = X - len(undisputed_top_true)

    tied_top_predicted = pred_qtr_val[pred_qtr_val == top_X_predicted.min()]
    tied_top_true = true_qtr_val[true_qtr_val == top_X_true.min()]

    error_in_top_true_ties = np.abs(tied_top_true - pred_qtr_val[tied_top_true.index]).sort_values(ascending=True)
    error_in_top_pred_ties = np.abs(true_qtr_val[tied_top_predicted.index] - tied_top_predicted).sort_values(
        ascending=True)
    top_true_tied_geoids = error_in_top_true_ties.iloc[:num_true_ties].index
    top_pred_tied_geoids = error_in_top_pred_ties.iloc[:num_tied_spots].index

    best_possible_top_true_geoids = pd.Index.union(undisputed_top_true.index, top_true_tied_geoids)
    best_possible_top_pred_geoids = pd.Index.union(undisputed_top_predicted.index, top_pred_tied_geoids)

    # True values of GEOIDS with highest actual deaths. If ties, finds tied locations that match preds best
    best_possible_true = true_qtr_val[best_possible_top_true_geoids]
    best_possible_pred = true_qtr_val[best_possible_top_pred_geoids]
    assert (len(best_possible_true) == X)
    assert (len(best_possible_pred) == X)

    best_possible_absolute = np.abs(best_possible_true.sum() - best_possible_pred.sum())
    best_possible_ratio = np.abs(best_possible_pred).sum() / np.abs(best_possible_true).sum()

    bootstrapped_tied_indices = np.random.choice(tied_top_predicted.index, (1000, num_tied_spots))
    bootstrapped_all_indices = [pd.Index.union(undisputed_top_predicted.index,
                                               bootstrap_index) for bootstrap_index in bootstrapped_tied_indices]

    
    import pdb;pdb.set_trace()
    bootstrapped_absolute = np.mean([np.abs(top_X_true.sum() - true_qtr_val[indices].sum())
                                     for indices in bootstrapped_all_indices])
    bootstrapped_ratio = np.mean([np.abs(true_qtr_val[indices]).sum() / np.abs(top_X_true).sum()
                                  for indices in bootstrapped_all_indices])

    return best_possible_absolute, best_possible_ratio, bootstrapped_absolute, bootstrapped_ratio